In [1]:
from pymilvus import connections, CollectionSchema, FieldSchema, DataType, Collection
from pymilvus import utility
from dotenv import load_dotenv
from langchain_community.vectorstores import Milvus
from langchain_openai import OpenAIEmbeddings
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_openai import ChatOpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
import hashlib
import json
import re
import os


In [2]:
load_dotenv()
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)


In [4]:
milvus = Milvus( embeddings, collection_name="collection_9", connection_args={"host": "localhost", "port": "19530"})


In [93]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory


retriever_doc_prompt = ChatPromptTemplate.from_messages(
                [("system", "Here:\\n\\n{context}")]
            )
cdc = create_stuff_documents_chain(llm=llm, prompt=retriever_doc_prompt)

retriever = milvus.as_retriever()

template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.
If the chat history can answer the follow up question, prepend context to the standalone question.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
template = """Chat History:
{chat_history}
Follow Up Input: {question}
Always return :Chat History  and follow up input"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(template)

prompt = PromptTemplate.from_template(template)
llm2 = ChatOpenAI(model_name="gpt-3.5-turbo")
question_generator_chain = LLMChain(llm=llm2, prompt=prompt)

template = """
  
  
  Having this context:
  {context}


  And this previous chat history: 
  {chat_history}

   Answer the following question: 
  {query}
  """

promptA = PromptTemplate(input_variables=["chat_history", "query", "context"], template=template)

crc = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, verbose=False, rephrase_question=False, condense_question_prompt=CONDENSE_QUESTION_PROMPT, combine_docs_chain_kwargs={"prompt": promptA})



In [83]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

chat_history_tuples = [HumanMessage(content='HOla me llamo juan. que hay en pasaia?'), 
                       AIMessage(content='En Pasaia, se celebra el Pasaia Itsas Festibala, un festival marítimo que incluye la visita a barcos de gran porte, desfiles de embarcaciones de madera, música, baile, y una gran variedad de actividades relacionadas con la cultura marítima. También hay espectáculos callejeros, como la compañía de títeres Kilika, que presenta un universo de títeres de gran escala.')]
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "Hola soy aritz y tengo 41 años."}, {"output": "hola Aritz"})

result = crc.invoke({'chat_history': memory.buffer_as_messages, 'query': "Sabes mi nombre?", 'question': "Sabes mi nombre?"})




> Entering new LLMChain chain...
Prompt after formatting:
Chat History:

Human: Hola soy aritz y tengo 41 años.
Assistant: hola Aritz
Follow Up Input: Sabes mi nombre?
Always return :Chat History  and follow up input

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

  
  
  Having this context:
  Ohiko Motora habitual
Festibaleko Motora Festival
Jatetxeak / Restaurantes
Bidegorria / Paseo peatonal 
Botikinak / Botiquines
Ontzi bisitagarriak / Barcos visitablesInteresguneak / Lugares de interés  INFOgunea
Lehiatila / Taquilla
Parada de BUS geltokia
Parada TREN Geltokia
Parada TAXI Geltokia
Komunak / Baños

Ongi etorri,
Aita Mari!
GKEAK AITA MARI ONTZIA IREKIKO DU ETA NAHI 
DUTEN GUZTIEK BISITATU AHAL IZANGO DUTE. 
Itsasontzia bisitatzea aukera bat da Mediterraneo 
Erdialdeko errealitatera hurbiltzeko eta Europan 
bizitza baketsu bat eraikitzen saiatzean milaka 
pertsonak aurre egin behar dioten bidegabeke -
ri

{'chat_history': [HumanMessage(content='Hola soy aritz y tengo 41 años.'),
  AIMessage(content='hola Aritz')],
 'query': 'Sabes mi nombre?',
 'question': 'Sabes mi nombre?',
 'answer': 'Sí, tu nombre es Aritz.'}

In [94]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

memory = ConversationBufferMemory(return_messages=True)



In [98]:
pregunta = "Tengo 40 años. Sabes mi edad?"
result = crc.invoke({'chat_history': memory.buffer_as_messages, 'query': pregunta, 'question': pregunta}, memory=memory)

print(result["answer"])
memory.save_context({"input": pregunta}, {"output": result["answer"]})

No, no sé tu edad.


In [133]:
from langchain.prompts import PromptTemplate

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, output_key='answer')

#If questions are asked where there is no relevant context available (previous chat history us valid information for you.), simply respond with "I don't know. Please ask a question relevant to the documents"

template = """
Responde a las preguntas basadas en el contexto o historial de chat dado.
<<HISTORIAL>>
{chat_history}

<<CONTEXTO>>
Context: {context}

Nueva pregunta: {question}
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "question"], template=template
)

# Create the custom chain
chain = ConversationalRetrievalChain.from_llm(
        llm=llm, retriever=retriever, memory=memory,
        return_source_documents=False,
        verbose=True,
        combine_docs_chain_kwargs={'prompt': prompt})


In [135]:

chain("Hola soy aritz. Tienes info del barco aita mari?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Responde a las preguntas basadas en el contexto o historial de chat dado.
<<HISTORIAL>>


<<CONTEXTO>>
Context: Ongi etorri,
Aita Mari!
GKEAK AITA MARI ONTZIA IREKIKO DU ETA NAHI 
DUTEN GUZTIEK BISITATU AHAL IZANGO DUTE. 
Itsasontzia bisitatzea aukera bat da Mediterraneo 
Erdialdeko errealitatera hurbiltzeko eta Europan 
bizitza baketsu bat eraikitzen saiatzean milaka 
pertsonak aurre egin behar dioten bidegabeke -
riara hurbiltzeko, eta haien ezinbesteko misio 
humanitarioarekin elkarlanean aritzeko.
Aita Mari, lehen lerroko lekukoaz gain, herrita -
rrentzako informazio, kontzientziazio eta eral -
daketarako plataforma ere izango da.LA ONG ABRIRÁ EL AITA MARI Y SERÁ VISITABLE 
POR TODAS LAS PERSONAS QUE LO DESEEN. 
La visita al barco es una oportunidad para acer -
carse a la realidad del Mediterráneo Central y la 
injusticia a la que se enfrentan miles de personas 
al intentar co

{'question': 'Hola soy aritz. Tienes info del barco aita mari?',
 'chat_history': [HumanMessage(content='Hola soy aritz. Tienes info del barco aita mari?'),
  AIMessage(content='Kaixo Aritz! Bai, badakizue Aita Mari itsasontziaren informazioa. Aita Mari itsasontzia, Mediterraneo erdialdean laguntza ematen duen ontzia da, eta Pasaia Itsas Festibala ontzi-flotako kide izango da. Ontzi Museo Ekoaktiboa da, eta San Pedroko kaian bisitagarria izango da, 11:00etatik 20:00etara doakoa. Aita Mari ere plataforma da informazio, kontzientziazio eta eraldaketarako. Gainera, itsasontziak bisitatzea ere aukera dago, eta beste tamaina handiko itsasontziak ere bisitagarriak izango dira, hala nola Grayhound, Le Biche, Phoenix eta Recouvrance. Espero izan dut lagungarria izatea!')],
 'answer': 'Kaixo Aritz! Bai, badakizue Aita Mari itsasontziaren informazioa. Aita Mari itsasontzia, Mediterraneo erdialdean laguntza ematen duen ontzia da, eta Pasaia Itsas Festibala ontzi-flotako kide izango da. Ontzi Muse

In [142]:
chain("Hay actividades para menores?")



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Hola soy aritz. Tienes info del barco aita mari?
Assistant: Kaixo Aritz! Bai, badakizue Aita Mari itsasontziaren informazioa. Aita Mari itsasontzia, Mediterraneo erdialdean laguntza ematen duen ontzia da, eta Pasaia Itsas Festibala ontzi-flotako kide izango da. Ontzi Museo Ekoaktiboa da, eta San Pedroko kaian bisitagarria izango da, 11:00etatik 20:00etara doakoa. Aita Mari ere plataforma da informazio, kontzientziazio eta eraldaketarako. Gainera, itsasontziak bisitatzea ere aukera dago, eta beste tamaina handiko itsasontziak ere bisitagarriak izango dira, hala nola Grayhound, Le Biche, Phoenix eta Recouvrance. Espero izan dut lagungarria izatea!
Human: Y ahora recuerdas mi nombre?
Assistant: Sí, tu nombre es Aritz.
Human: Te agradezco que te acuerdes de mi. QUe

{'question': 'Hay actividades para menores?',
 'chat_history': [HumanMessage(content='Hola soy aritz. Tienes info del barco aita mari?'),
  AIMessage(content='Kaixo Aritz! Bai, badakizue Aita Mari itsasontziaren informazioa. Aita Mari itsasontzia, Mediterraneo erdialdean laguntza ematen duen ontzia da, eta Pasaia Itsas Festibala ontzi-flotako kide izango da. Ontzi Museo Ekoaktiboa da, eta San Pedroko kaian bisitagarria izango da, 11:00etatik 20:00etara doakoa. Aita Mari ere plataforma da informazio, kontzientziazio eta eraldaketarako. Gainera, itsasontziak bisitatzea ere aukera dago, eta beste tamaina handiko itsasontziak ere bisitagarriak izango dira, hala nola Grayhound, Le Biche, Phoenix eta Recouvrance. Espero izan dut lagungarria izatea!'),
  HumanMessage(content='Y ahora recuerdas mi nombre?'),
  AIMessage(content='Sí, tu nombre es Aritz.'),
  HumanMessage(content='Te agradezco que te acuerdes de mi. QUe tipo de misiones acomete el Aita Mari?'),
  AIMessage(content='El Aita Mari 

In [131]:
from langchain.chains import RetrievalQAWithSourcesChain
prompt_template = '''
Answer my questions based on your knowledge and our older conversation. Do not make up answers.
If you do not know the answer to a question, just say "I don't know".

{context}

Given the following conversation and a follow up question, answer the question.

{chat_history}

question: {question}
'''

PROMPT = PromptTemplate(
            template=prompt_template, input_variables=["context", "chat_history", "question"]
        )

memory = ConversationBufferMemory(
                                    memory_key="chat_history",
                                    max_len=50,
                                    return_messages=True,
                                    output_key='answer'
                                )
# this time it was required to specify output_key in memory.

chain = RetrievalQAWithSourcesChain.from_chain_type(
                        llm=llm,
                        chain_type="stuff",
                        retriever=retriever,
                        memory=memory,
                        verbose=True,
                    )

In [132]:
chain(PROMPT.format(question="HOla soy Aritz. Sabes algo del barco Aita Mari?", chat_history=memory.chat_memory.messages, context=''))



> Entering new RetrievalQAWithSourcesChain chain...

> Finished chain.


{'question': '\nAnswer my questions based on your knowledge and our older conversation. Do not make up answers.\nIf you do not know the answer to a question, just say "I don\'t know".\n\n\n\nGiven the following conversation and a follow up question, answer the question.\n\n[]\n\nquestion: HOla soy Aritz. Sabes algo del barco Aita Mari?\n',
 'chat_history': [HumanMessage(content='\nAnswer my questions based on your knowledge and our older conversation. Do not make up answers.\nIf you do not know the answer to a question, just say "I don\'t know".\n\n\n\nGiven the following conversation and a follow up question, answer the question.\n\n[]\n\nquestion: HOla soy Aritz. Sabes algo del barco Aita Mari?\n'),
  AIMessage(content="I don't know.")],
 'answer': "I don't know.",
 'sources': ''}